# CAPSTONE PROJECT: DECISION TREES

## Question: Based on the data, can we predict what category of apps will be used?

Given the data, we are going to predict the category of application the device will use in the future. The assumption is that if a device uses a certain type of application in the past, the user of the device will be more inclined to use an application of the same type in the future. 

# Data Preparation

Data was pulled in from 5 files: ApiCalls, ApiNames, AppInteractivity, DeviceCensus, and StoreSearch. The tables were concatenated and merged into a FinalData DataFrame that I wanted to use which included Categories: DateId, nDeviceId, appSessionGuid, AppName, InFocusDurationMS, EngagementDurationMS, appName an Category. In order to run the dataframe through the DecisionTreeClassfier, a new class MultiLabelEncoder was created.

In [ ]:
# -*- coding: utf-8 -*-
"""
Spyder Editor

This is a temporary script file.
"""

import pandas as pd
ApiCalls = pd.read_table("../../Desktop/Data/ApiCalls.tsv")
ApiNames=pd.read_table("../../Desktop/Data/ApiNames.tsv")
AppInteractivity = pd.read_table("../../Desktop/Data/AppInteractivity.tsv")
DeviceCensus = pd.read_table("../../Desktop/Data/DeviceCensus.tsv")
StoreSearch=pd.read_table("../../Desktop/Data/StoreSearch.tsv")

#whittle the dataframes for concatnation
AppInter_adj = pd.DataFrame(AppInteractivity[["DateId","nDeviceId","appSessionGuid","AppName","InFocusDurationMS","EngagementDurationMS"]])
SS_adj = pd.DataFrame(StoreSearch[["Category","appName"]])
SS_adj.columns=["Category","AppName"]

#Concat
SourceData = pd.concat([ApiCalls,AppInter_adj,ApiNames],axis=0,ignore_index = True, sort=False)

#Merge
FinalData = pd.merge(SS_adj,SourceData, on=["AppName"])


#Based on previous data, the device will use an x type application tomorrow
#Using Decision Trees
import sklearn.datasets as datasets
from sklearn.model_selection import train_test_split
import numpy as np
from sklearn import preprocessing

#Create a MultiColumLabelEncoder
class MultiColumnLabelEncoder(preprocessing.LabelEncoder):
    """
    Wraps sklearn LabelEncoder functionality for use on multiple columns of a
    pandas dataframe.

    """
    def __init__(self, columns=None):
        self.columns = columns

    def fit(self, dframe):
        # if columns are provided, iterate through and get `classes_`
        if self.columns is not None:
            # ndarray to hold LabelEncoder().classes_ for each
            # column; should match the shape of specified `columns`
            self.all_classes_ = np.ndarray(shape=self.columns.shape,
                                           dtype=object)
            self.all_encoders_ = np.ndarray(shape=self.columns.shape,
                                            dtype=object)
            for idx, column in enumerate(self.columns):
                # fit LabelEncoder to get `classes_` for the column
                le = preprocessing.LabelEncoder()
                le.fit(dframe.loc[:, column].values)
                # append the `classes_` to our ndarray container
                self.all_classes_[idx] = (column,
                                          np.array(le.classes_.tolist(),
                                                  dtype=object))
                # append this column's encoder
                self.all_encoders_[idx] = le
        else:
            # no columns specified; assume all are to be encoded
            self.columns = dframe.iloc[:, :].columns
            self.all_classes_ = np.ndarray(shape=self.columns.shape,
                                           dtype=object)
            for idx, column in enumerate(self.columns):
                le = preprocessing.LabelEncoder()
                le.fit(dframe.loc[:, column].values)
                self.all_classes_[idx] = (column,
                                          np.array(le.classes_.tolist(),
                                                  dtype=object))
                self.all_encoders_[idx] = le
        return self

    def fit_transform(self, dframe):
        # if columns are provided, iterate through and get `classes_`
        if self.columns is not None:
            # ndarray to hold LabelEncoder().classes_ for each
            # column; should match the shape of specified `columns`
            self.all_classes_ = np.ndarray(shape=self.columns.shape,
                                           dtype=object)
            self.all_encoders_ = np.ndarray(shape=self.columns.shape,
                                            dtype=object)
            self.all_labels_ = np.ndarray(shape=self.columns.shape,
                                          dtype=object)
            for idx, column in enumerate(self.columns):
                # instantiate LabelEncoder
                le = preprocessing.LabelEncoder()
                # fit and transform labels in the column
                dframe.loc[:, column] =\
                    le.fit_transform(dframe.loc[:, column].values)
                # append the `classes_` to our ndarray container
                self.all_classes_[idx] = (column,
                                          np.array(le.classes_.tolist(),
                                                  dtype=object))
                self.all_encoders_[idx] = le
                self.all_labels_[idx] = le
        else:
            # no columns specified; assume all are to be encoded
            self.columns = dframe.iloc[:, :].columns
            self.all_classes_ = np.ndarray(shape=self.columns.shape,
                                           dtype=object)
            for idx, column in enumerate(self.columns):
                le = preprocessing.LabelEncoder()
                dframe.loc[:, column] = le.fit_transform(
                        dframe.loc[:, column].values)
                self.all_classes_[idx] = (column,
                                          np.array(le.classes_.tolist(),
                                                  dtype=object))
                self.all_encoders_[idx] = le
        return dframe

    def transform(self, dframe):
        ##Transform labels to normalized encoding.
        if self.columns is not None:
            for idx, column in enumerate(self.columns):
                dframe.loc[:, column] = self.all_encoders_[
                    idx].transform(dframe.loc[:, column].values)
        else:
            self.columns = dframe.iloc[:, :].columns
            for idx, column in enumerate(self.columns):
                dframe.loc[:, column] = self.all_encoders_[idx]\
                    .transform(dframe.loc[:, column].values)
        return dframe.loc[:, self.columns].values

    def inverse_transform(self, dframe):
        ###Transform labels back to original encoding.
        if self.columns is not None:
            for idx, column in enumerate(self.columns):
                dframe.loc[:, column] = self.all_encoders_[idx]\
                    .inverse_transform(dframe.loc[:, column].values)
        else:
            self.columns = dframe.iloc[:, :].columns
            for idx, column in enumerate(self.columns):
                dframe.loc[:, column] = self.all_encoders_[idx]\
                    .inverse_transform(dframe.loc[:, column].values)
        return dframe
    
#Pick out Mixed type data frames
FeaturePicker = pd.DataFrame(FinalData[["Category","DateId","nDeviceId"]])        
# get `object` columns
df_object_columns = FeaturePicker.iloc[:, :].select_dtypes(include=['object']).columns

#instantiate MultiColumnLabelEncoder
mcle = MultiColumnLabelEncoder(columns=df_object_columns)

#fit to FeaturePicker
mcle.fit(FeaturePicker)

#transform the FeaturePicker
mcle.transform(FeaturePicker)
print(FeaturePicker.head())

#inverse data
#mcle.inverse_transform(FinalData)
#X=Feature
#Y=Label

y=FeaturePicker.Category.to_frame()
X_train, X_test, y_train, y_test=train_test_split(FeaturePicker.iloc[:,1:],y,test_size=0.33,random_state=42)


#Sklearn will generate a decision tree for your dataset using an optimized version of the CART algorithm when you run the following code
from sklearn.tree import DecisionTreeClassifier
dtree = DecisionTreeClassifier()
dtree.fit(X_train,y_train)

#metrics
import sklearn.metrics as met
y_pred = dtree.predict(X_test)
print(met.classification_report(y_test,y_pred))
#Decode the arrays back into string

#Graph the Tree
from sklearn.externals.six import StringIO  
from IPython.display import Image  
from sklearn.tree import export_graphviz
import pydotplus
dot_data = StringIO()
export_graphviz(dtree, out_file=dot_data,  
                filled=True, rounded=True,
                special_characters=True)
graph = pydotplus.graph_from_dot_data(dot_data.getvalue())  
Image(graph.create_png())


C:\Users\hochung\AppData\Local\Continuum\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2785: DtypeWarning: Columns (101,122,123) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


   Category      DateId  nDeviceId
0         0  20180420.0       79.0
1         0  20180420.0       79.0
2         0  20180420.0       79.0
3         0  20180420.0       79.0
4         0  20180420.0       79.0


C:\Users\hochung\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


             precision    recall  f1-score   support

          0       0.51      0.42      0.46      6007
          1       0.65      0.56      0.60      8505
          2       0.76      0.14      0.24       175
          3       0.45      0.45      0.45      4932
          4       0.56      0.39      0.46    112919
          5       0.64      0.71      0.67    398394
          6       0.61      0.61      0.61    112972
          7       0.00      0.00      0.00       193
          8       0.00      0.00      0.00       106
          9       0.66      0.35      0.46       359
         10       0.43      0.05      0.09       190
         11       0.61      0.61      0.61    196603
         12       0.54      0.60      0.57      5889
         13       0.55      0.51      0.53     80923
         14       0.48      0.47      0.48      7507
         15       0.64      0.52      0.57       956
         16       0.72      0.82      0.77   2161095
         17       0.58      0.54      0.56   

# Results 

In [ ]:
from IPython.display import Image
from IPython.core.display import HTML 
PATH = "C:\Users\hochung\Documents\Python Notebooks"
Image(filename = PATH + "Results_For_Capstone.png", width=100, height=100)